# Quotes for swaps

In [ ]:
#| default_exp quote

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from functools import reduce
from typing import List, Union, Tuple
from eth_abi.packed import encode_packed
from dataclasses import dataclass
from sugar.token import Token
from sugar.pool import LiquidityPoolForSwap
from typing import Optional

In [ ]:
#| export 

# magic numbers
QUOTER_STABLE_POOL_FILLER, QUOTER_VOLATILE_POOL_FILLER = 2097152, 4194304

@dataclass
class PreparedRoute:
    types: List[str]; values: List[Union[str, int, bool]]

    @property
    def encoded(self) -> bytes: return encode_packed(self.types, self.values)

def pack_path(path: List[Tuple[LiquidityPoolForSwap, bool]], for_swap: bool = False) -> PreparedRoute:
    is_v2_swap = for_swap and any(pool.is_basic for pool, _ in path)
    types, values = reduce(lambda s, pool: s + pool, [["address", "int24"] for i in range(len(path))], []) + ["address"], []
    for node in path:
        pool, reversed = node
        token0, token1 = pool.token0_address if not reversed else pool.token1_address, pool.token1_address if not reversed else pool.token0_address
        if pool.type > 0: filler = pool.type
        else: filler =  QUOTER_STABLE_POOL_FILLER if pool.is_stable else QUOTER_VOLATILE_POOL_FILLER
        if len(values) == 0: values = [token0, filler, token1]
        else: values += [filler, token1]
    
    if is_v2_swap:
        types = list(map(lambda t: "bool" if t == "int24" else t, types))
        for i in range(len(values)):
            if values[i] == QUOTER_STABLE_POOL_FILLER: values[i] = True
            elif values[i] == QUOTER_VOLATILE_POOL_FILLER: values[i] = False

    return PreparedRoute(types=types, values=values)
  

@dataclass
class QuoteInput:
    from_token: Token; to_token: Token; path: List[Tuple[LiquidityPoolForSwap, bool]]; amount_in: int

    def to_tuple(self) -> tuple: return (self.from_token, self.to_token, tuple(self.path), self.amount_in, self.amount_out)

    @staticmethod
    def from_tuple(t: tuple) -> "QuoteInput":
        from_token, to_token, path, amount_in, amount_out = t
        return QuoteInput(from_token=from_token, to_token=to_token, path=list(path), amount_in=amount_in, amount_out=amount_out)
    
    @property
    def route(self) -> PreparedRoute: return pack_path(self.path)

    @property
    def route_for_swap(self) -> PreparedRoute: return pack_path(self.path, for_swap=True)

@dataclass
class Quote:
    input: QuoteInput; amount_out: int

    @property
    def from_token(self) -> Token: return self.input.from_token
    @property
    def to_token(self) -> Token: return self.input.to_token
    @property
    def path(self) -> List[Tuple[LiquidityPoolForSwap, bool]]: return self.input.path
    @property
    def amount_in(self) -> int: return self.input.amount_in


## Superswap 🐝 quote 

In [ ]:
#| export


@dataclass(frozen=True)
class SuperswapQuote:
    from_token: Token
    to_token: Token
    from_bridge_token: Token
    to_bridge_token: Token
    amount_in: int
    origin_quote: Optional[Quote] = None
    destination_quote: Optional[Quote] = None

    @property
    def amount_out(self) -> int:
        return self.destination_quote.amount_out if self.destination_quote else self.bridged_amount

    @staticmethod
    def bridge_quote(from_token: Token, to_token: Token, amount: int) -> 'SuperswapQuote':
        return SuperswapQuote(from_token=from_token, to_token=to_token, from_bridge_token=from_token, to_bridge_token=to_token, amount_in=amount)

    @staticmethod
    def calc_bridged_amount(from_token: Token, from_bridge_token: Token, amount: int, origin_quote: Optional[Quote] = None) -> int:
        if from_token != from_bridge_token:
            assert origin_quote is not None, "origin_quote must be set"
            return origin_quote.amount_out
        else: return amount

    @property
    def bridged_amount(self) -> int: 
        return SuperswapQuote.calc_bridged_amount(
            from_token=self.from_token,
            from_bridge_token=self.from_bridge_token, amount=self.amount_in, origin_quote=self.origin_quote
        )

    @property
    def is_bridge(self) -> bool: return self.from_token == self.from_bridge_token and self.to_token == self.to_bridge_token

## Tests

In [ ]:
from fastcore.test import test_eq

In [ ]:
path = [
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=200, token0_address='0x4200000000000000000000000000000000000006', token1_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40'), False),
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0x02A130b6D35611bC2D90e20f2ceA45431c0A9a8d', type=-1, token0_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', token1_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607'), False),
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xf7a73e1995030B0198f8d6e528a1c42ACEf90a4c', type=0, token0_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', token1_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'), False)
]

rt = pack_path(path)

test_eq(rt.types, ['address', 'int24', 'address', 'int24', 'address', 'int24', 'address'])
test_eq(rt.values, ['0x4200000000000000000000000000000000000006', 200, '0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', 4194304, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607', 2097152, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'])
test_eq(rt.encoded.hex(), "42000000000000000000000000000000000000060000c86c84a8f1c29108f47a79964b5fe888d4f4d0de404000007f5c764cbc14f9669b88837ca1490cca17c316072000009560e827af36c94d2ac33a39bce1fe78631088db")

# test reversed version

path = [
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=200, token0_address='0x4200000000000000000000000000000000000006', token1_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40'), True),
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0x02A130b6D35611bC2D90e20f2ceA45431c0A9a8d', type=-1, token0_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', token1_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607'), False),
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xf7a73e1995030B0198f8d6e528a1c42ACEf90a4c', type=0, token0_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', token1_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'), False)
]

rt = pack_path(path)

test_eq(rt.types, ['address', 'int24', 'address', 'int24', 'address', 'int24', 'address'])
test_eq(rt.values, ['0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', 200, '0x4200000000000000000000000000000000000006', 4194304, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607', 2097152, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'])
test_eq(rt.encoded.hex(), "6c84a8f1c29108f47a79964b5fe888d4f4d0de400000c842000000000000000000000000000000000000064000007f5c764cbc14f9669b88837ca1490cca17c316072000009560e827af36c94d2ac33a39bce1fe78631088db")

v2_nodes = [
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xF1046053aa5682b4F9a81b5481394DA16BE5FF6b', type=-1, token0_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', token1_address='0x920Cf626a271321C151D027030D5d08aF699456b'), False),
    (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xF1046053aa5682b4F9a81b5481394DA16BE5FF6b', type=-1, token0_address='0x920Cf626a271321C151D027030D5d08aF699456b', token1_address='0x4200000000000000000000000000000000000042'), False)
]

rt = pack_path(v2_nodes, for_swap=True)
test_eq(rt.types, ['address', 'bool', 'address', 'bool', 'address'])
test_eq(rt.values, [
    "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db",
    False,
    "0x920Cf626a271321C151D027030D5d08aF699456b",
    False,
    "0x4200000000000000000000000000000000000042",
])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()